In [6]:
import cv2
import csv
import numpy as np
import glob
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib auto

Using matplotlib backend: TkAgg


In [7]:
df = pd.read_csv('dataset.csv')

In [20]:
n = df.to_numpy()
plt.imshow(n[0,1:].reshape(28,28))